# Tutorial to invoke LIME explainers via aix360

There are two ways to use [LIME](https://github.com/marcotcr/lime) explainers after installing aix360:
- [Approach 1  (aix360 style)](#approach1): LIME explainers can be invoked in a manner similar to other explainer algorithms in aix360 via the implemented wrapper classes.
- [Approach 2 (original style)](#approach2): Since LIME comes pre-installed in aix360, the explainers can simply be invoked directly.

This notebook showcases both these approaches to invoke LIME. The notebook is based on the following example from the original LIME tutorial: https://marcotcr.github.io/lime/tutorials/Lime%20-%20multiclass.html

## Approach 1  (aix360 style)
<a id='approach1'></a>
- Note the import statement related to LimeTextExplainer

In [1]:
from __future__ import print_function
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics

# Importing LimeTextExplainer (aix360 sytle)
from aix360.algorithms.lime import LimeTextExplainer

In [2]:
# Supress jupyter warnings if required for cleaner output
import warnings
warnings.simplefilter('ignore')

### Fetching data, training a classifier

In [3]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
# making class names shorter
class_names = [x.split('.')[-1] if 'misc' not in x else '.'.join(x.split('.')[-2:]) for x in newsgroups_train.target_names]
class_names[3] = 'pc.hardware'
class_names[4] = 'mac.hardware'

In [4]:
print(','.join(class_names))

atheism,graphics,ms-windows.misc,pc.hardware,mac.hardware,x,misc.forsale,autos,motorcycles,baseball,hockey,crypt,electronics,med,space,christian,guns,mideast,politics.misc,religion.misc


In [5]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [6]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)
nb.fit(train_vectors, newsgroups_train.target)

MultinomialNB(alpha=0.01)

In [7]:
pred = nb.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='weighted')

0.8350184193998174

### Explaining predictions using lime

In [8]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, nb)

In [9]:
print(c.predict_proba([newsgroups_test.data[0]]).round(3))

[[0.001 0.01  0.003 0.047 0.006 0.002 0.003 0.521 0.022 0.008 0.025 0.
  0.331 0.003 0.006 0.    0.003 0.    0.001 0.009]]


In [10]:
limeexplainer = LimeTextExplainer(class_names=class_names)
print(type(limeexplainer))

<class 'aix360.algorithms.lime.lime_wrapper.LimeTextExplainer'>


In [11]:
idx = 1340
# aix360 style for explaining input instances
exp = limeexplainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6, labels=[0, 17])
print('Document id: %d' % idx)
print('Predicted class =', class_names[nb.predict(test_vectors[idx]).reshape(1,-1)[0,0]])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

Document id: 1340
Predicted class = atheism
True class: atheism


In [12]:
print ('Explanation for class %s' % class_names[0])
print ('\n'.join(map(str, exp.as_list(label=0))))
print ()
print ('Explanation for class %s' % class_names[17])
print ('\n'.join(map(str, exp.as_list(label=17))))

Explanation for class atheism
('Caused', 0.26238077066590293)
('Rice', 0.13518306047536863)
('Genocide', 0.13057135539747292)
('scri', -0.08935717895685585)
('owlnet', -0.0866086512211189)
('Semitic', -0.08450085710232705)

Explanation for class mideast
('fsu', -0.05515369453392734)
('Theism', -0.054747456372004545)
('Luther', -0.0470171558437435)
('Caused', -0.03726710714147818)
('jews', 0.03390695559844592)
('PBS', 0.032592761622772665)


## Approach 2 (original style)
<a id='approach2'></a>
- Note the import statement related to LimeTextExplainer

In [13]:
from __future__ import print_function
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics

# Importing LimeTextExplainer (original style)
from lime.lime_text import LimeTextExplainer

In [14]:
# Supress jupyter warnings if required for cleaner output
import warnings
warnings.simplefilter('ignore')

### Fetching data, training a classifier

In [15]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
# making class names shorter
class_names = [x.split('.')[-1] if 'misc' not in x else '.'.join(x.split('.')[-2:]) for x in newsgroups_train.target_names]
class_names[3] = 'pc.hardware'
class_names[4] = 'mac.hardware'

In [16]:
print(','.join(class_names))

atheism,graphics,ms-windows.misc,pc.hardware,mac.hardware,x,misc.forsale,autos,motorcycles,baseball,hockey,crypt,electronics,med,space,christian,guns,mideast,politics.misc,religion.misc


In [17]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

In [18]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB(alpha=.01)
nb.fit(train_vectors, newsgroups_train.target)

MultinomialNB(alpha=0.01)

In [19]:
pred = nb.predict(test_vectors)
sklearn.metrics.f1_score(newsgroups_test.target, pred, average='weighted')

0.8350184193998174

### Explaining predictions using lime

In [20]:
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, nb)

In [21]:
print(c.predict_proba([newsgroups_test.data[0]]).round(3))

[[0.001 0.01  0.003 0.047 0.006 0.002 0.003 0.521 0.022 0.008 0.025 0.
  0.331 0.003 0.006 0.    0.003 0.    0.001 0.009]]


In [22]:
explainer = LimeTextExplainer(class_names=class_names)
print(type(explainer))

<class 'lime.lime_text.LimeTextExplainer'>


In [23]:
idx = 1340
# LIME original style for explaining input instances
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6, labels=[0, 17])
print('Document id: %d' % idx)
print('Predicted class =', class_names[nb.predict(test_vectors[idx]).reshape(1,-1)[0,0]])
print('True class: %s' % class_names[newsgroups_test.target[idx]])

Document id: 1340
Predicted class = atheism
True class: atheism


In [24]:
print ('Explanation for class %s' % class_names[0])
print ('\n'.join(map(str, exp.as_list(label=0))))
print ()
print ('Explanation for class %s' % class_names[17])
print ('\n'.join(map(str, exp.as_list(label=17))))

Explanation for class atheism
('Caused', 0.25632751113248026)
('Rice', 0.14626237282926857)
('Genocide', 0.13133950204310213)
('owlnet', -0.0905864608741577)
('scri', -0.08526752642948768)
('Semitic', -0.08287079245253995)

Explanation for class mideast
('fsu', -0.05714575197470851)
('Theism', -0.05248892887178565)
('Luther', -0.05034586274638904)
('Rice', -0.03470048019899408)
('jews', 0.03372175158157365)
('PBS', 0.030816731955572742)
